# Dridex Process Pattern
Detects typical Dridex process patterns

## Rule Content
```
- title: Dridex Process Pattern
  id: e6eb5a96-9e6f-4a18-9cdd-642cfda21c8e
  status: experimental
  description: Detects typical Dridex process patterns
  references:
  - https://app.any.run/tasks/993daa5e-112a-4ff6-8b5a-edbcec7c7ba3
  author: Florian Roth
  date: 2019/01/10
  tags:
  - attack.defense_evasion
  - attack.privilege_escalation
  - attack.t1055
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      CommandLine: '*\svchost.exe C:\Users\\*\Desktop\\*'
    selection2:
      ParentImage: '*\svchost.exe*'
      CommandLine:
      - '*whoami.exe /all'
      - '*net.exe view'
    condition: 1 of them
  falsepositives:
  - Unlikely
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:*\\svchost.exe\ C\:\\Users\\*\\Desktop\\* OR (process_parent_path.keyword:*\\svchost.exe* AND process_command_line.keyword:(*whoami.exe\ \/all OR *net.exe\ view)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()